In [1]:
# imports
from astropy.io import fits
import matplotlib.pyplot as plt
# import matplotlib as mpl
import plotting as myP
import numpy as np
import csv 
import match

In [2]:
# path variables 
path_cat = 'D:\IfA REU\Catalogs'
path_fig = 'D:\IfA REU\IfA-REU-Project\Figures'
path_csv = 'D:\IfA REU\IfA-REU-Project\Lists'

# Read AGN Lists From CSV

In [3]:
# read from a given csv file and return a list of its contents 
def ReadFile(filename) : 
    # initialize list
    file = []

    # open csv file
    with open(path_csv+filename, newline='') as csv_file : 
        reader = csv.reader(csv_file, delimiter=',', quotechar='"')
        # output file into list for easy access
        for row in reader :
            file.append(row)

    # print column names 
    print(filename, '\tcolumns: ', file[0])

    # return an array of type int with column names clipped off
    return( np.array(file[1:], dtype=int) )


In [6]:
## read AGN ID and index from csv files
# COSMOS 2020
agn_irSel20 = ReadFile('\Match_COSMOS2020_IRAGN_to_COSMOS2020.csv')
agn_xrSel20 = ReadFile('\Match_Chandra2016_XrayAGN_to_COSMOS2020.csv')
# Chandra 2016
agn_irSel16 = ReadFile('\Match_COSMOS2020_IRAGN_to_Chandra2016.csv')
agn_xrSel16 = ReadFile('\Match_Chandra2016_XrayAGN_to_Chandra2016.csv')

## transpose arrays to get columns
# COSMOS 2020
agn_irSel20_id, agn_irSel20_i = agn_irSel20.T
agn_xrSel20_id, agn_xrSel20_i = agn_xrSel20.T
# Chandra 2016
agn_irSel16_id, agn_irSel16_i = agn_irSel16.T
agn_xrSel16_id, agn_xrSel16_i = agn_xrSel16.T

\Match_COSMOS2020_IRAGN_to_COSMOS2020.csv 	columns:  ['ID', 'i']
\Match_Chandra2016_XrayAGN_to_COSMOS2020.csv 	columns:  ['ID', 'i']
\Match_COSMOS2020_IRAGN_to_Chandra2016.csv 	columns:  ['ID', 'i']
\Match_Chandra2016_XrayAGN_to_Chandra2016.csv 	columns:  ['ID', 'i']


In [9]:
# print info
print('Number of IR AGN in COSMOS 2020:\t',    len(agn_irSel20_id))
print('Number of Xray AGN in COSMOS 2020:\t',  len(agn_xrSel20_id))
print('Number of IR AGN in Chandra 2016:\t',   len(agn_irSel16_id))
print('Number of Xray AGN in Chandra 2016:\t', len(agn_xrSel16_id))

Number of IR AGN in COSMOS 2020:	 2287
Number of Xray AGN in COSMOS 2020:	 2176
Number of IR AGN in Chandra 2016:	 746
Number of Xray AGN in Chandra 2016:	 2232


In [ ]:
# # test: left and right must match
# print(agn_irSel20[0][0], '==', agn_irSel20_id[0])
# print(agn_irSel20[0][1], '==', agn_irSel20_i[0])
# print(agn_xrSel20[0][0], '==', agn_xrSel20_id[0])
# print(agn_xrSel20[0][1], '==', agn_xrSel20_i[0])

# Get Data From Catalogs

In [ ]:
# open COSMOS 2020 catalog file and get data 
inf20 = fits.open(path_cat+'\COSMOS2020_CLASSIC_R1_v2.0_master.fits')
data20 = inf20[1].data
cols20 = inf20[1].columns
inf20.close()

In [ ]:
# get IRAC data (unit = 'uJy')
ch1_data  = np.array(data20['SPLASH_CH1_FLUX']) # 3.5686 um
print('Ch1 read')
ch2_data  = np.array(data20['SPLASH_CH2_FLUX']) # 4.5067 um
print('Ch2 read')
ch3_data  = np.array(data20['SPLASH_CH3_FLUX']) # 5.7788 um
print('Ch3 read')
ch4_data  = np.array(data20['SPLASH_CH4_FLUX']) # 7.9958 um 
print('Ch4 read')

In [ ]:
# open COSMOS 2016 catalog file and get data 
inf16 = fits.open(path_cat+'\chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits')
data16 = inf16[1].data
cols16 = inf16[1].columns
inf16.close()

In [ ]:
# get data arrays from columns
L0510_data = np.array(data16['Lx_0510'])        # x-ray luminosity in the 0.5-10 KeV band
print('L0510 read')

# Isolate Data By Selection Method

In [ ]:
# match agn ids
key_ir, key_xr = match.match(agn_irSel20_id, agn_xrSel20_id)

# initialize
mask_ir = np.ones(agn_irSel20_id.size, dtype=bool)
mask_xr = np.ones(agn_xrSel20_id.size, dtype=bool)

# set matched index to false
mask_ir[key_ir] = False
mask_xr[key_xr] = False

# remove xray selected agn from ir selected list
agn_irSel20_id_ONLY = agn_irSel20_id[mask_ir]
agn_irSel20_i_ONLY  = agn_irSel20_i[mask_ir]

# remove ir selected agn from xray selected list 
agn_xrSel20_id_ONLY = agn_xrSel20_id[mask_xr]
agn_xrSel20_i_ONLY  = agn_xrSel20_i[mask_xr]

# create list of ir/xray match
agn20_id_BOTH = agn_irSel20_id[key_ir]
agn20_i_BOTH  = agn_irSel20_i[key_ir]

In [ ]:
# print info 
print('Number of IR only:\t',   len(agn_irSel20_i_ONLY))
print('Number of Xray only:\t', len(agn_xrSel20_i_ONLY))
print('Number of matches:\t',   len(agn20_i_BOTH))

In [ ]:
# returns the IRAC channels only at indeces i 
def SelectIRAC(i):
    # get data
    a = ch1_data[i]
    b = ch2_data[i]
    c = ch3_data[i]
    d = ch4_data[i]

    # mask bad values 
    mask = (a > 0) & (b > 0) & (c > 0) & (d > 0)

    # apply mask 
    af = a[mask]
    bf = b[mask]
    cf = c[mask]
    df = d[mask] 

    # print info
    print('Col \t Min \t\t Max \t\t Count')
    print('-------------------------------------------------------------')
    print('CH1 \t', min(af), '\t', max(af), '\t', len(af))
    print('CH2 \t', min(bf), '\t', max(bf), '\t', len(bf))
    print('CH3 \t', min(cf), '\t', max(cf), '\t', len(cf))
    print('CH4 \t', min(df), '\t', max(df), '\t', len(df))
    print('-------------------------------------------------------------')
    # return data 
    return af,bf,cf,df

In [ ]:
# filter data by selection method
ch1_ir, ch2_ir, ch3_ir, ch4_ir = SelectIRAC(agn_irSel20_i_ONLY)
ch1_xr, ch2_xr, ch3_xr, ch4_xr = SelectIRAC(agn_xrSel20_i_ONLY)
ch1_ix, ch2_ix, ch3_ix, ch4_ix = SelectIRAC(agn20_i_BOTH)

# Plot IRAC Colors With Exclusive Selection Lists

In [ ]:
# set standard plot format
myP.SetStyle()

In [ ]:
# calculate colors for ir 
x_ir = np.log10( ch3_ir / ch1_ir ) # x = log10 ( f_5.6um / f_3.6um )
y_ir = np.log10( ch4_ir / ch2_ir ) # y = log10 ( f_8.0um / f_4.5 um )

# calculate colors for xray 
x_xr = np.log10( ch3_xr / ch1_xr ) # x = log10 ( f_5.6um / f_3.6um )
y_xr = np.log10( ch4_xr / ch2_xr ) # y = log10 ( f_8.0um / f_4.5 um )

# calculate colors for matched ir/xray
x_ix = np.log10( ch3_ix / ch1_ix ) # x = log10 ( f_5.6um / f_3.6um )
y_ix = np.log10( ch4_ix / ch2_ix ) # y = log10 ( f_8.0um / f_4.5 um )

In [ ]:
# plot data
plt.plot(x_ir, y_ir, '.', c=myP.c_ir,       markersize=1, label='IR selected')
plt.plot(x_xr, y_xr, 'x', c=myP.c_xray,     markersize=1, label='X-ray selected')
plt.plot(x_ix, y_ix, '*', c=myP.c_ir_xray,  markersize=1, label='both IR and X-ray')

# draw box
myP.PlotDonleyWedge()

# customize
plt.legend(markerscale=8)
plt.title('IR and X-ray selected AGN from COSMOS sources') 

# set axis size and save
plt.axis([-1.5,2.5,-1.5,2.5]) 
plt.savefig(path_fig+'\COSMOS_IR-Xray_selection_FULL.png',bbox_inches ="tight",pad_inches=0.2,facecolor='w',edgecolor='w')
plt.axis([-0.7,1.0,-0.7,1.0]) 
plt.savefig(path_fig+'\COSMOS_IR-Xray_selection_ZOOM.png',bbox_inches ="tight",pad_inches=0.2,facecolor='w',edgecolor='w')

# display
plt.show()

# Plot IRAC Colors With Xray Color Bar